## Libs

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string
import re
from sklearn.model_selection import train_test_split

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\josev\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\josev\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Dataset Info

In [2]:
df = pd.read_csv(r'D:\CoDes\FACTCKBR.tsv', delimiter='\t')

In [3]:
print(df.head())
print(df.info())

                                                 URL                  Author  \
0  https://aosfatos.org/noticias/governo-bolsonar...  https:www.aosfatos.org   
1  https://aosfatos.org/noticias/nao-e-miriam-lei...  https:www.aosfatos.org   
2  https://aosfatos.org/noticias/nao-e-verdade-qu...  https:www.aosfatos.org   
3  https://aosfatos.org/noticias/nao-e-de-mulher-...  https:www.aosfatos.org   
4  https://aosfatos.org/noticias/e-falso-que-refo...  https:www.aosfatos.org   

  datePublished                                      claimReviewed  \
0    2019-07-22  Espaço dedicado para os eleitores do Bolsonaro...   
1    2019-07-22  Vos apresento a funcionária protegida pela Glo...   
2    2019-07-22  Você sabia que Benedita da Silva já foi embaix...   
3    2019-07-18  QUEM  A MULHER QUE EMPURROU O PADRE MARCELO ROSSI   
4    2019-07-17  As crueldades da Reforma da PrevidênciaPessoas...   

                                          reviewBody  \
0  Publicações que circulam nas redes soci

In [4]:
# Para obter o número de valores únicos em cada coluna
valores_unicos = df.nunique()
valores_unicos

URL                 984
Author                3
datePublished       772
claimReviewed      1291
reviewBody          774
title               984
ratingValue           9
bestRating            3
alternativeName      18
dtype: int64

In [5]:
valores_ausentes_por_coluna = df.isna().sum()
print(valores_ausentes_por_coluna)

URL                 0
Author              0
datePublished       0
claimReviewed      13
reviewBody         12
title               0
ratingValue         4
bestRating          0
alternativeName     4
dtype: int64


In [6]:
valores_unicos = df['alternativeName'].unique()
valores_unicos

array(['falso', 'distorcido', 'impreciso', 'exagerado', 'insustentável',
       'verdadeiro', 'outros', 'Falso', 'Subestimado', 'Verdadeiro',
       'Exagerado', 'Impossível provar', 'Discutível', 'Sem contexto',
       nan, 'Distorcido', 'De olho', 'Verdadeiro, mas',
       'Ainda é cedo para dizer'], dtype=object)

## Pre-Processing

In [7]:
# Dropar linhas onde alguma das colunas é NaN
df = df.dropna()
valores_ausentes_por_coluna = df.isna().sum()
print(valores_ausentes_por_coluna)

URL                0
Author             0
datePublished      0
claimReviewed      0
reviewBody         0
title              0
ratingValue        0
bestRating         0
alternativeName    0
dtype: int64


In [8]:
# Inicializar o stemmer e stopwords
stop_words = set(stopwords.words('portuguese'))
stemmer = PorterStemmer()

In [9]:
def clean_text(text):
    # Remover números
    text = re.sub(r'\d+', '', text)
    # Converter para minúsculas
    text = text.lower()
    # Remover pontuação
    text = text.translate(str.maketrans('', '', string.punctuation))

    return text

# Função para processar texto
def process_text(text):
    text = clean_text(text)
    tokens = [word for word in word_tokenize(text) if word not in stop_words]
    stemmed = [stemmer.stem(word) for word in tokens]
    return ' '.join(stemmed)

In [10]:
columns_to_process = [
    'URL', 
    'Author', 
    'claimReviewed', 
    'reviewBody', 
    'title', 
    'alternativeName'
]

In [11]:
for column in columns_to_process:
    # Aplicar o pré-processamento em cada coluna
    texts = df[column].astype(str).apply(process_text)
    
    # Converter para DataFrame e adicionar ao dicionário de colunas processadas
    df[column] = pd.DataFrame(texts, index=df.index)

df

,URL,Author,datePublished,claimReviewed,reviewBody,title,ratingValue,bestRating,alternativeName
0,httpsaosfatosorgnoticiasgovernobolsonaronaosus...,httpswwwaosfatosorg,2019-07-22,espaço dedicado eleitor bolsonaro explicarem r...,publicaçõ circulam rede sociai veja aqui engan...,governo bolsonaro suspendeu distribuição insul...,1.0,5,falso
1,httpsaosfatosorgnoticiasnaoemiriamleitaoquemse...,httpswwwaosfatosorg,2019-07-22,apresento funcionária protegida globo miriam t...,foto treinamento defesa contra assalto banco o...,miriam leitão segura fuzil lado carlo lamarca ...,1.0,5,falso
2,httpsaosfatosorgnoticiasnaoeverdadequebenedita...,httpswwwaosfatosorg,2019-07-22,sabia benedita silva embaixadora news york ing...,falsa informação deputada feder benedita silva...,verdad benedita silva embaixadora brasil eua fato,1.0,5,falso
3,httpsaosfatosorgnoticiasnaoedemulherqueempurro...,httpswwwaosfatosorg,2019-07-18,mulher empurr padr marcelo rossi,modificada digitalment incluir filtro foto ser...,mulher empurr padr marcelo rossi foto filtro ‘...,1.0,5,falso
4,httpsaosfatosorgnoticiasefalsoquereformatirade...,httpswwwaosfatosorg,2019-07-17,crueldad reforma previdênciapessoa síndrome do...,publicação circula rede sociai afirma reforma ...,falso reforma tira pessoa deficiência direito ...,1.0,5,falso
...,...,...,...,...,...,...,...,...,...
1308,httpspiauifolhauolcombrlupaverificamoscarmendo...,httpspiauifolhauolcombrlupa,2018-05-17 15:23:40,cármen lúcia raquel dogd acabam pedir prisão e...,empti,verificamo pgr cármen lúcia pediram prisão ime...,4.0,6,falso
1309,httpspiauifolhauolcombrlupaverificamosmarinacarta,httpspiauifolhauolcombrlupa,2018-05-16 17:14:19,marina assina carta prol casamento homossexu,empti,verificamo marina assin carta prol casamento h...,4.0,6,falso
1310,httpspiauifolhauolcombrlupaverificamosdocinoce...,httpspiauifolhauolcombrlupa,2018-05-16 12:00:30,tribun divulga documento prova inocência lula,empti,verificamo decisão tribun df prova inocência lula,4.0,6,falso
1311,httpspiauifolhauolcombrlupaverificamossurtogri...,httpspiauifolhauolcombrlupa,2018-05-15 13:00:20,direção hospit clínica paulo preocupada nova g...,empti,verificamo vacina contra gripe causa surto mor...,4.0,6,falso


## Double Check at Data

In [14]:
valores_unicos = df['alternativeName'].unique()
valores_unicos

array(['falso', 'distorcido', 'impreciso', 'exagerado', 'insustentável',
       'verdadeiro', 'outro', 'subestimado', 'impossível provar',
       'discutível', 'contexto', 'olho', 'ainda cedo dizer'], dtype=object)

In [15]:
# Dicionário de mapeamento
mapping = {
    'falso': 'falso',
    'distorcido': 'falso',
    'impreciso': 'falso',
    'exagerado': 'falso',
    'insustentável': 'falso',
    'verdadeiro': 'verdadeiro',
    'outro': 'inconclusivo',
    'subestimado': 'inconclusivo',
    'impossível provar': 'inconclusivo',
    'discutível': 'inconclusivo',
    'contexto': 'inconclusivo',
    'olho': 'inconclusivo',
    'ainda cedo dizer': 'inconclusivo'
}

# Aplicando o mapeamento
df['alternativeName'] = df['alternativeName'].replace(mapping)

print(df)

                                                    URL  \
0     httpsaosfatosorgnoticiasgovernobolsonaronaosus...   
1     httpsaosfatosorgnoticiasnaoemiriamleitaoquemse...   
2     httpsaosfatosorgnoticiasnaoeverdadequebenedita...   
3     httpsaosfatosorgnoticiasnaoedemulherqueempurro...   
4     httpsaosfatosorgnoticiasefalsoquereformatirade...   
...                                                 ...   
1308  httpspiauifolhauolcombrlupaverificamoscarmendo...   
1309  httpspiauifolhauolcombrlupaverificamosmarinacarta   
1310  httpspiauifolhauolcombrlupaverificamosdocinoce...   
1311  httpspiauifolhauolcombrlupaverificamossurtogri...   
1312  httpspiauifolhauolcombrlupaverificamossurtogri...   

                           Author        datePublished  \
0             httpswwwaosfatosorg           2019-07-22   
1             httpswwwaosfatosorg           2019-07-22   
2             httpswwwaosfatosorg           2019-07-22   
3             httpswwwaosfatosorg           2019-07-18   
4

In [16]:
# Para obter o número de valores únicos em cada coluna
valores_unicos = df.nunique()
valores_unicos

URL                 969
Author                3
datePublished       765
claimReviewed      1269
reviewBody          771
title               971
ratingValue           9
bestRating            3
alternativeName       3
dtype: int64

## Spliting and Vetorizing

In [ ]:
train, test = train_test_split(df, test_size=0.2, random_state=42)

In [13]:
# # Converter para TF-IDF
# vectorizer = TfidfVectorizer()
# # Normalizar os dados
# normalizer = Normalizer()


# train_vectors = vectorizer.fit_transform(train['coluna'])
# train_vectors = normalizer.fit_transform(train_vectors)
# test_vectors = vectorizer.transform(test['coluna'])
# test_vectors = normalizer.transform(test_vectors)